# Guide to tHMM

In [54]:
import numpy as np
import scipy.stats as sp
import matplotlib.pyplot as plt

### Synthesizing Cells (not required by the user)

In [55]:
from lineage.CellVar import CellVar as c
from lineage.CellVar import _double

In [56]:
T = np.array([[1.0, 0.0],
              [0.0, 1.0]])
    
parent_state = 1
parent_cell = c(state=parent_state, left=None, right=None, parent=None, gen=1)
left_cell, right_cell = parent_cell._divide(T)

In [57]:
print(left_cell, parent_cell.left)


 Generation: 2, State: 1, Observation: This cell has no observations to report. 
 Generation: 2, State: 1, Observation: This cell has no observations to report.


In [58]:
print(right_cell, parent_cell.right)


 Generation: 2, State: 1, Observation: This cell has no observations to report. 
 Generation: 2, State: 1, Observation: This cell has no observations to report.


## Creating a synthetic lineage (required by the user) "Two State Model"

In [59]:
from lineage.LineageTree import LineageTree
from lineage.StateDistribution import StateDistribution, get_experiment_time

### Creating a lineage and setting the full lineage (unpruned) as the one to be used

The required probabilities are those that define the tree and act of state switching. This process works by first creating a hidden tree of empty cells. Empty cells are those that have their states set but do not have any observations attached to them. We then draw as many observations from each state distribution and assign those observations to those cells. The $\pi$ and $T$ parameters are easy to define. The number of states is $k$. We require for $\pi$ a $k\times 1$ list of probabilities. These probabilities must add up to $1$ and they should be either in a $1$-dimensional list or a $1$-dimensional numpy array. The $T$ parameter should be a square numpy matrix of size $k\times k$. The rows are the states in which we are transitioning from and the columns are the states in which we are transitioning to. Each row of $T$ should sum to $1$. The columns need not sum to $1$.

In [102]:
# pi: the initial probability vector
pi = np.array([0.6, 0.4], dtype="float")

# T: transition probability matrix
T = np.array([[0.85, 0.15],
              [0.15, 0.85]])

The emission matrix $E$ is a little more complicated to define because this is where the user has complete freedom in defining what type of observation they care about. In particular, the user has to first begin with defining what observation he or she will want in their cells in their synthetic images. For example, if one is observing kinematics or physics, they might want to use Gaaussian distribution observations. In defining the random variables, the user will pull from a Gaussian distribution based on the mean and standard deviation of the different states he or she picks. They can also utilize the Gaussian probability distribution to define the likelihood as well. Furthermore, they can build an analytical estimator for their state distributions that yield the parameter estimates when given a list of observations. Finally, the user can also define a prune rule, which is essentially a boolean function that inspects a cell's observations and returns True if the cell's subtree (all the cells that are related to the cell in question and are of older generation) is to be pruned or False if the cell is safe from pruning. In the Gaussian example, a user can remove a cell's subtree if its observation is higher or lower than some fixed value.

We have already built, as an example, and as bioengineers, a model that resembles lineage trees. In our synthetic model, our emissions are multivariate. This first emission is a Bernoulli observation, $0$ implying death and $1$ implying division. The second and third emissions are continuous and are from exponential and gamma distributions respectively. Though these can be thought of cell lifetime's or periods in a certain cell phase, we want the user to know that these values can really mean anything and they are completely free in choosing what the emissions and their values mean. We define ways to calculate random variables for these multivariate observations and likelihoods of an observations. We also provide as a prune rule, keeping with the cell analogy, that if a cell has a $0$ in its Bernoulli observation, then its subtree is pruned from the full lineage tree. Though this will obviously introduce bias into estimation, we keep both the full tree and the pruned tree in the lineage objects, in the case a user would like to see the effects of analyzing on one versus the other.

Ultimately, $E$ is defined as a $k\times 1$ size list of state distribution objects. These distribution objects are rich in what they can already do, and a user can easily add more to their functionality. They only need to be instantiated by what parameters define that state's distribution.

In [61]:
# E: states are defined as StateDistribution objects

# State 0 parameters "Resistant"
state0 = 0
bern_p0 = 0.99
gamma_loc = 0
gamma_a0 = 20
gamma_scale0 = 5

# State 1 parameters "Susceptible"
state1 = 1
bern_p1 = 0.88
gamma_a1 = 15
gamma_scale1 = 1

state_obj0 = StateDistribution(state0, bern_p0, gamma_a0, gamma_loc, gamma_scale0)
state_obj1 = StateDistribution(state1, bern_p1, gamma_a1, gamma_loc,  gamma_scale1)

E = [state_obj0, state_obj1]

The final required parameters are more obvious. The first is the desired number of cells one would like in their full unpruned lineage tree. This can be any number. The lineage tree is built 'from left to right'. What this means is that, we construct the binary tree by going to the left-most cell, dividing then walking through the generation. For example, if someone requested for

In [62]:
desired_num_cells = 2**7 - 1 
prune_boolean = False # To get the full tree

In [63]:
lineage1 = LineageTree(pi, T, E, desired_num_cells, desired_experiment_time=1000, prune_condition='both', prune_boolean=False)
print(lineage1)

This tree is NOT pruned. It is made of 2 states.
 For each state in this tree: 
 	 There are 65 cells of state 0, 
 	 There are 62 cells of state 1.
 This UNpruned tree has 127 many cells in total


### Obtaining how long the experiment ran by checking the time length of the longest branch

In [64]:
longest_branch_time = get_experiment_time(lineage1)
print(longest_branch_time)

710.957277373252


### Estimation of distribution parameters using our estimators for full lineage

In [65]:
for state in range(lineage1.num_states):
    print("State {}:".format(state))
    print("                    estimated state", E[state].estimator(lineage1.lineage_stats[state].full_lin_cells_obs))
    print("original parameters given for state", E[state])

State 0:
                    estimated state State object w/ parameters: 0.9538461538447573, 26.008909508411826, 0.0, 3.750203910672563.
original parameters given for state State object w/ parameters: 0.99, 20, 0, 5.
State 1:
                    estimated state State object w/ parameters: 0.8548387096762748, 14.026058569272587, 0.0, 1.064891497886133.
original parameters given for state State object w/ parameters: 0.88, 15, 0, 1.


### Estimation of distribution parameters using our estimators for pruned lineage

In [66]:
for state in range(lineage1.num_states):
    print("State {}:".format(state))
    print("                    estimated state", E[state].estimator(lineage1.lineage_stats[state].pruned_lin_cells_obs))
    print("original parameters given for state", E[state])

State 0:
                    estimated state State object w/ parameters: 0.9615384615366865, 25.955852053130364, 0.0, 3.6607775564332785.
original parameters given for state State object w/ parameters: 0.99, 20, 0, 5.
State 1:
                    estimated state State object w/ parameters: 0.8775510204066224, 13.173762218560649, 0.0, 1.1221592385061.
original parameters given for state State object w/ parameters: 0.88, 15, 0, 1.


### Analyzing our first full lineage

In [67]:
from lineage.Analyze import Analyze, accuracy, KL_analyze

X = [lineage1] # population just contains one lineage
states = [cell.state for cell in lineage1.output_lineage]
deltas, state_ptrs, all_states, tHMMobj, NF, LL = Analyze(X, 2) # find two states

In [68]:
all_states

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])]

In [69]:
def kl_divergence(p, q):
    """ Performs KL-divergence as:
        KL(P||Q) = Integral[ P(x) log(P(x)/Q(x)) ] for continuous distributions,
        and summation instead of integral, for discrete distributions. """
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))
import scipy

In [135]:
import random
# pi: the initial probability vector
pi = np.array([0.6, 0.4], dtype="float")

# T: transition probability matrix
T = np.array([[0.65, 0.35],
              [0.35, 0.65]])

gamma_a0List = [5.0, 10.0, 15.0, 12.0]
gamma_scale0List = [2.0, 2.0, 2.0, 3.3]
gamma_a1List = [23.0, 20.0, 17.0, 12.0]
gamma_scale1List = [3.0, 3.0, 3.0, 3.3]

gammaKL = []
acc = []
for i in range(4):
    state_obj0 = StateDistribution(state0, bern_p0, gamma_a0List[i], gamma_loc, gamma_scale0List[i])
    state_obj1 = StateDistribution(state1, bern_p1, gamma_a1List[i], gamma_loc,  gamma_scale1List[i])

    E = [state_obj0, state_obj1]
    lineageObj = LineageTree(pi, T, E, desired_num_cells=2**11 -1, desired_experiment_time=100000, prune_condition='both', prune_boolean=False)
    X = [lineageObj]
    states = [cell.state for cell in lineageObj.output_lineage]
    deltas, state_ptrs, all_states, tHMMobj, NF, LL = Analyze(X, 2)

    # find the accuracy
    temp = accuracy(tHMMobj, all_states)
    acc.append(temp[0])

    # find the KL divergence
    state0obs=[]
    state1obs=[]

    for indx, cell in enumerate(lineageObj.output_lineage):
        if all_states[0][indx] == 0:
            state0obs.append(cell.obs[1])
        elif all_states[0][indx] == 1:
            state1obs.append(cell.obs[1])
    p=scipy.stats.gamma.pdf(state0obs, a=gamma_a0List[i], loc=gamma_loc, scale=gamma_scale0List[i])
    q=scipy.stats.gamma.pdf(state1obs, a=gamma_a1List[i], loc=gamma_loc, scale=gamma_scale1List[i])

    size = min(p.shape[0], q.shape[0])
    if size == 0:
        raise ValueError('the number of cells predicted in one of the states is zero! ')
    else:
        pprime = random.sample(list(p), size)
        qprime = random.sample(list(q), size)
    gammaKL.append(kl_divergence(np.asarray(pprime), np.asarray(qprime)))

ValueError: the number of cells predicted in one of the states is zero! 

In [ ]:
print("acc", acc)
print("KL", gammaKL)

In [ ]:
plt.scatter(gammaKL, acc)

### Estimated Markov parameters ($\pi$, $T$, $E$)

In [111]:
print(tHMMobj.estimate.pi)

[1.00000000e+00 2.06741757e-14]


In [112]:
print(tHMMobj.estimate.T)

[[9.99363182e-01 6.36817730e-04]
 [1.00000000e+00 2.66312379e-10]]


In [77]:
for state in range(tHMMobj.numStates):
    print(tHMMobj.estimate.E[state])

State object w/ parameters: 0.9433317049340065, 9.827991561406423, 0.0, 3.061589374401238.
State object w/ parameters: 0.5, 10, 0, 1.


## Trying another lineage, this time pruning branches with ancestors that die

In [78]:
desired_num_cells = 2**12 -1 
prune_boolean = True # To get pruned tree

In [79]:
lineage2 = LineageTree(pi, T, E, desired_num_cells, prune_boolean)
print(lineage2)

This tree is pruned. It is made of 2 states.
 For each state in this tree: 
 	 There are 3 cells of state 0, 
 	 There are 0 cells of state 1.
 This pruned tree has 3 many cells in total


In [80]:
longest2 = get_experiment_time(lineage2)
print(longest2)

61.31041435850814


### Estimation of distribution parameters using our estimators for pruned lineage

In [81]:
for state in range(lineage2.num_states):
    print("State {}:".format(state))
    print("                    estimated state", E[state].estimator(lineage2.lineage_stats[state].pruned_lin_cells_obs))
    print("original parameters given for state", E[state])

State 0:
                    estimated state State object w/ parameters: 0.9999999999666667, 20.956382919953885, 0.0, 1.2891601003267479.
original parameters given for state State object w/ parameters: 0.99, 10.0, 0, 3.0.
State 1:
                    estimated state State object w/ parameters: 0.5, 10, 0, 1.
original parameters given for state State object w/ parameters: 0.88, 10.0, 0, 3.0.


### Analyzing a population of lineages

In [82]:
X = [lineage1, lineage2] # population just contains one lineage

deltas, state_ptrs, all_states, tHMMobj, NF, LL = Analyze(X, 2) # find two states

In [83]:
from lineage.Analyze import accuracy
accuracy(tHMMobj, all_states)

[48.818897637795274, 100.0]

### Estimated Markov parameters ($\pi$, $T$, $E$)

In [84]:
print(tHMMobj.estimate.pi)

[1.00000000e+00 5.03234369e-30]


In [85]:
print(tHMMobj.estimate.T)

[[1.00000000e+00 1.67684275e-14]
 [1.00000000e+00 1.55558022e-14]]


In [86]:
for state in range(tHMMobj.numStates):
    print(tHMMobj.estimate.E[state])

State object w/ parameters: 0.5, 10, 0, 1.
State object w/ parameters: 0.9076923076916804, 1.3839094374800456, 0.0, 40.83810528203561.


## Creating a synthetic lineage that has three states

Here we generate a lineage with three states, which would be 1) Susciptible 2) Middle State 3) Resistant. The aim here is to show the transition from susciptible to resistant state doesn't happen immediately, and there is a gradual transition which is modeled as a middle state. The point to be considered here is that transition from 1 to 3 or otherwise is not possible so the probability of these transitions are zero, and most likely the initial cells are in susciptible state.

**State 1**: Susceptible

**State 2**: Transition state

**State 3**: Resistant state


In [87]:
# pi: the initial probability vector
pi_3 = np.array([0.5, 0.25, 0.25])

# T: transition probability matrix
T_3 = np.array([[0.65, 0.35, 0.00],
                [0.20, 0.40, 0.40],
                [0.00, 0.10, 0.90]])

In [88]:
# E: states are defined as StateDistribution objects

# State 0 parameters "Susciptible"
state0 = 0
bern_p0 = 0.7
gamma_a0 = 5.0
gamma_scale0 = 1.0

# State 1 parameters "Middle state"
state1 = 1
bern_p1 = 0.85
gamma_a1 = 10.0
gamma_scale1 = 2.0

# State 2 parameters "Resistant"
state2 = 2
bern_p2 = 0.99
gamma_a2 = 15.0
gamma_scale2 = 3.0

state_obj0 = StateDistribution(state0, bern_p0, gamma_a0, gamma_loc, gamma_scale0)
state_obj1 = StateDistribution(state1, bern_p1, gamma_a1, gamma_loc, gamma_scale1)
state_obj2 = StateDistribution(state2, bern_p2, gamma_a2, gamma_loc, gamma_scale2)

E_3 = [state_obj0, state_obj1, state_obj2]

In [89]:
desired_num_cells = 2**13 - 1 
prune_boolean = False # To get the full tree

In [90]:
lineage3 = LineageTree(pi_3, T_3, E_3, desired_num_cells, prune_boolean)
print(lineage3)

This tree is pruned. It is made of 3 states.
 For each state in this tree: 
 	 There are 2 cells of state 0, 
 	 There are 1 cells of state 1, 
 	 There are 0 cells of state 2.
 This pruned tree has 3 many cells in total


In [91]:
longest3 = get_experiment_time(lineage3)
print(longest3)

36.1291671805797


### Estimation of distribution parameters using our estimators for full lineage (3 state)

In [92]:
for state in range(lineage3.num_states):
    print("State {}:".format(state))
    print("estimated state", E_3[state].estimator(lineage3.lineage_stats[state].full_lin_cells_obs))
    print("estimated state", E_3[state].estimator(lineage3.lineage_stats[state].pruned_lin_cells_obs))
    print("true_____ state", E_3[state])

State 0:
estimated state State object w/ parameters: 0.7030859049207334, 10, 0.0, 1.
estimated state State object w/ parameters: 0.99999999995, 10, 0.0, 1.
true_____ state State object w/ parameters: 0.7, 5.0, 0, 1.0.
State 1:
estimated state State object w/ parameters: 0.8541051388068099, 10.238403209621048, 0.0, 1.9613181419902246.
estimated state State object w/ parameters: 0.9999999999, 10, 0.0, 1.
true_____ state State object w/ parameters: 0.85, 10.0, 0, 2.0.
State 2:
estimated state State object w/ parameters: 0.98886582374031, 14.451801152057795, 0.0, 3.1225141309744884.
estimated state State object w/ parameters: 0.5, 10, 0, 1.
true_____ state State object w/ parameters: 0.99, 15.0, 0, 3.0.


### Analyzing a three state lineage

In [93]:
X = [lineage3] # population just contains one lineage

deltas, state_ptrs, all_states, tHMMobj, NF, LL = Analyze(X, 3) # find three states

In [94]:
accuracy(tHMMobj, all_states)

[100.0]

### Estimated Markov parameters ($\pi$, $T$, $E$)

In [95]:
print(tHMMobj.estimate.pi)

[2.75369175e-57 1.00000000e+00 6.89241946e-64]


In [96]:
print(tHMMobj.estimate.T)

[[7.18783188e-01 2.81216812e-01 2.83875567e-61]
 [7.18783188e-01 2.81216812e-01 1.59095999e-57]
 [7.18783188e-01 2.81216812e-01 9.54553193e-60]]
